<a href="https://colab.research.google.com/github/jakeengstrom3/SampleImageClassification/blob/master/Copy_of_Image_Classification_Transfer_Learning_from_Images_on_Local_Disk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
#!ls /mydrive

# Image Classification Transfer learning example with dataset from local disk




This Colab follows closely the https://www.tensorflow.org/lite/models/modify/model_maker/image_classification example, but with some small changes to upload custom images instead of downloading the example dataset

# Step 0: Prerequisites

In [ ]:
import os

In [ ]:
!pip install -q tflite-model-maker

In [ ]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

# Step 1: Loading data from local disk into TF ImageDataset object
**Before running, go to the folder icon and drag a zipped dataset into /content/.**

The dataset must have images properly sorted into labeled sub directories. Each sub directory will correspond to a new class in the models head layer. For example, make sure all images of plastic bottles are in the directory dataset/plastic_bottles/. 

In [ ]:
!unzip -q Recyclables.zip

Set image path to the name of the uploaded data folder

In [ ]:
image_path = 'Recyclables'

Using tflife_model_maker/image_classifer/Dataloader to load the data from folder. This will automatically label the images with the name of thier resident directory. 

In [ ]:
data = DataLoader.from_folder(image_path)

Showing an example of 25 images to make sure the data was loaded in properly

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

## Train test splitting
Here we are splitting the data into training, validation, and test data, with a 0.8 to 0.1 to 0.1 split, respectively. 

In [ ]:
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
print(type(train_data))

# Step 2: Customizing the TF model

Choose pretrained model to customize. Options using ModelMaker are: 
* 'efficientnet_lite0',
*'efficientnet_lite1',
*'efficientnet_lite2',
*'efficientnet_lite3',
*'efficientnet_lite4',
*'mobilenet_v2',
*'resnet_50'

In [ ]:
model_spec='mobilenet_v2'

In [ ]:
model = image_classifier.create(
  train_data, 
  validation_data=validation_data,
  model_spec=model_spec,
  epochs=2
)

# Step 3: Evalutate the model

In [ ]:
loss, accuracy = model.evaluate(test_data)


In [ ]:
y_test = []
y_pred = []

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
missed = 0
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  y_pred.append(predict_label)
  y_test.append(test_data.index_to_label[label.numpy()])
  if color == 'red':
    missed+=1
    """
    plt.figure(figsize=IMAGE_SIZE, dpi=100)
    plt.axis("off")
    plt.imshow(image_np_with_detections)
    plt.show()
    """
    plt.figure(figsize=(4,2), dpi=60)
    plt.imshow(image.numpy(), cmap=plt.cm.gray)
    plt.xlabel('Predicted: %s' % predict_label + 
              '\nActual: %s' % test_data.index_to_label[label.numpy()] 
               , color= 'red')
    
plt.show()
print("total = ", missed)

# Export the model

In [ ]:
x = model.export(export_dir='.')

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('android.tflite')

In [ ]:
model.evaluate(validation_data)
model.evaluate(test_data)

In [ ]:
"""x = [i[0] for i in predicts]
final_labels = [i[0] for i in x]"""


# Metrics

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix #
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support #
from sklearn.metrics import classification_report #
from sklearn.metrics import ConfusionMatrixDisplay #

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [False, True])

cm_display.plot()
plt.show()

In [ ]:
print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))

In [ ]:
print(accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))